**sEMG CLASSIFIER**

In [ ]:
# This file contains the necessary code to train the sEMG signals classifier, through a ResNet-18

In [ ]:
# Use the following lines if you want to use Google Colab
# We presume you created a folder "i2dl" within your main drive folder, and put the exercise there.
# NOTE: terminate all other colab sessions that use GPU!
# NOTE 2: Make sure the correct exercise folder (e.g exercise_10) is given.

from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/adl4r'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

In [1]:
import torch
import torchvision
print(f"PyTorch version Installed: {torch.__version__}\nTorchvision version Installed: {torchvision.__version__}\n")
if not torch.__version__.startswith("1.11"):
    print("you are using an another version of PyTorch. We expect PyTorch 1.11.0. You may continue using your version but it"
          " might cause dependency and compatibility issues.")
if not torchvision.__version__.startswith("0.12"):
    print("you are using an another version of torchvision. We expect torchvision 0.12. You can continue with your version but it"
          " might cause dependency and compatibility issues.")

c:\Users\jgual\anaconda3\envs\your_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version Installed: 1.13.1
Torchvision version Installed: 0.14.1+cpu

you are using an another version of PyTorch. We expect PyTorch 1.11.0. You may continue using your version but it might cause dependency and compatibility issues.
you are using an another version of torchvision. We expect torchvision 0.12. You can continue with your version but it might cause dependency and compatibility issues.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import sklearn.metrics

%load_ext autoreload
%autoreload 2
%matplotlib inline

ModuleNotFoundError: No module named 'sklearn'

In [3]:
# DATASET AND DATALOADER

from sEMG_dataset import sEMGDataset
from torch.utils.data import DataLoader

transform = transforms.Compose([
    # Apply any necessary transformations
    transforms.ToTensor(),
])

# Generation of the training, validation, and testing datasets
train_dataset = sEMGDataset("./semg_dataset/spectrograms_concat",mode = 'train', split = {'train': 0.8, 'val': 0.1, 'test': 0.1}, transform = transform)
val_dataset = sEMGDataset("./semg_dataset/spectrograms_concat",mode = 'val', split = {'train': 0.8, 'val': 0.1, 'test': 0.1}, transform = transform)
test_dataset = sEMGDataset("./semg_dataset/spectrograms_concat",mode = 'test', split = {'train': 0.8, 'val': 0.1, 'test': 0.1}, transform = transform)

# Generation of the training, validation, and testing datasets
train_dataloader = DataLoader(train_dataset , batch_size=10, shuffle=True, num_workers=2, drop_last=False)
val_dataloader = DataLoader(val_dataset , batch_size=10, shuffle=True, num_workers=2, drop_last=False)
test_dataloader = DataLoader(test_dataset , batch_size=10, shuffle=True, num_workers=2, drop_last=False)

In [8]:
# Evaluation of samples contained in each dataset
print('Training length: ' + str(len(test_dataset)))
print('Validation length: ' + str(len(val_dataset)))
print('Test length: ' + str(len(test_dataset)))
print('Sample shape: ' + str(train_dataset[0][0].shape))

Training length: 270
Validation length: 270
Test length: 270
Sample shape: torch.Size([6, 480, 640])


In [9]:
# Classifier (ResNet18)
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Model definition
model = models.resnet18(pretrained = False)

# Adjust the classification (channels in and channels out)
num_channels = 6  # channels in
model.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_classes = 6 # channels out
num_features = model.fc.in_features  # Número de características en la capa de clasificación actual
model.fc = nn.Linear(num_features, num_classes)

# Loss function to be used for classification CE Loss
criterion = nn.CrossEntropyLoss()

# Optimizer definition: Adam SGD
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum= 0.9)

c:\Users\jgual\anaconda3\envs\your_env\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
c:\Users\jgual\anaconda3\envs\your_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# TRAINING OUR CLASSIFIER

# Number of epochs to be trained
n_epochs = 45

# Set device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Utilizar GPU si está disponible
model = model.to(device)
criterion = criterion.to(device)

# Initializing the list for storing the loss and accuracy
train_loss_history = [] # loss
train_acc_history = [] # accuracy
val_loss_history = [] # loss validation
val_acc_history = [] # accuracy validation


for epoch in range(n_epochs):
    
    # TRAINING
    model.train()
    running_loss = 0.0
    correct = 0.0
    
    for imgs, labels in train_dataloader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        # Reset the parameter gradients  for the current  minibatch iteration 
        optimizer.zero_grad()
        y_pred = model(imgs)             # Perform a forward pass on the network with inputs
        loss = criterion(y_pred, labels) # calculate the loss with the network predictions and ground Truth
        loss.backward()             # Perform a backward pass to calculate the gradients
        optimizer.step()            # Optimize the network parameters with calculated gradients

        
        # Accumulate the loss and calculate the accuracy of predictions
        running_loss += loss.item()
        _, preds = torch.max(y_pred, 1) #convert output probabilities of each class to a singular class prediction
        correct += torch.sum(preds == labels).item()

    #print and store statistics
    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct / len(train_dataset)
    train_loss_history.append(epoch_loss)
    train_acc_history.append(epoch_accuracy)

    print("[Epoch %d] --> Train loss: %.3f Train accuracy: %.2f %%" % (epoch+1, epoch_loss, epoch_accuracy*100))


    # VALIDATION
    model.eval()
    running_loss = 0.0
    correct = 0.0
   
    for imgs, labels in val_dataloader:
        with torch.no_grad():
            imgs = imgs.to(device)
            labels = labels.to(device)

            y_pred = model(imgs)             # Perform a forward pass on the network with inputs
            loss = criterion(y_pred, labels) # calculate the loss with the network predictions and ground Truth
            
            # Accumulate the loss and calculate the accuracy of predictions
            running_loss += loss.item()
            _, preds = torch.max(y_pred, 1) #convert output probabilities of each class to a singular class prediction
            correct += torch.sum(preds == labels).item()

    #print and store statistics
    epoch_loss = running_loss / len(val_dataset)
    epoch_accuracy = correct / len(val_dataset)
    val_loss_history.append(epoch_loss)
    val_acc_history.append(epoch_accuracy)

    print("[Epoch %d] --> Val loss: %.3f Val accuracy: %.2f %%" % (epoch+1, epoch_loss, epoch_accuracy*100))
    print('--------------')
    

print('FINISH.')

In [ ]:
# PLOT TRAIN AND VALIDATION LOSS/ACCURACY
import matplotlib.pyplot as plt

# Loss
plt.plot(range(1, n_epochs+1), train_loss_history, label='Train Loss')
plt.plot(range(1, n_epochs+1), val_loss_history, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Accuracy
plt.plot(range(1, n_epochs+1), train_acc_history, label='Train Accuracy')
plt.plot(range(1, n_epochs+1), val_acc_history, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# TESTING
model.eval()
running_loss = 0.0
correct = 0.0
acc = 0.0
recall = 0.0    
precision = 0.0
f1 = 0.0
confusion = np.zeros((6, 6))


for imgs, labels in test_dataloader:
    with torch.no_grad():
        imgs = imgs.to(device)
        labels = labels.to(device)

        y_pred = model(imgs)             # Perform a forward pass on the network with inputs
        loss = criterion(y_pred, labels) # calculate the loss with the network predictions and ground Truth
        
        # Accumulate the loss and calculate the accuracy of predictions
        running_loss += loss.item()
        _, preds = torch.max(y_pred, 1) #convert output probabilities of each class to a singular class prediction
        correct += torch.sum(preds == labels).item()
        preds = preds.data.cpu().numpy()
        targets = labels.data.cpu().numpy()
        recall += sklearn.metrics.recall_score(targets.flatten(), preds.flatten(),
                                               average="weighted", zero_division=0)
        precision += sklearn.metrics.precision_score(targets.flatten(), preds.flatten(),
                                                     average="weighted", zero_division=0)
        f1 += sklearn.metrics.f1_score(targets.flatten(), preds.flatten(),
                                       average="weighted", zero_division=0)
        
        for j in range(len(imgs)):
            confusion[preds[j], targets[j]] += 1


#print and store statistics
epoch_loss = running_loss / len(test_dataset)
epoch_accuracy = correct / len(test_dataset)

f = open("test_2_45e.txt", "w")
print(confusion, file=f)
print("Test accuracy: %.2f %%" % (epoch_accuracy*100), file=f)
print('Recall: {:.4f}'.format(recall / len(test_dataloader)), file=f)
print('Precision: {:.4f}'.format(precision / len(test_dataloader)), file=f)
print('f1: {:.4f}'.format(f1 / len(test_dataloader)), file=f)

f.close()